# BioExplorer - Vasculature
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3
import seaborn as sns

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

morphology_realism_level = be.MORPHOLOGY_REALISM_LEVEL_SOMA 
vasculature_realism_level = be.VASCULATURE_REALISM_LEVEL_NONE

### Vasculature

In [ ]:
vasculature_assembly_name = 'Vasculature'
vasculature_population_name = 'vasculature'

o = Vector3(334.274978637695, 0, 598.822242736816)
d = Vector3(0.0, 1.0, 0.0)
r = 400.0
filter = '%s.length(%s.cross(array[%f-x,%f-y,%f-z], array[%f::double precision, %f, %f])) < %f' % (vasculature_population_name, vasculature_population_name, o.x, o.y, o.z, d.x, d.y, d.z, r)

be.remove_assembly(vasculature_assembly_name)
vasculature_assembly = be.add_assembly(vasculature_assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=vasculature_assembly_name,
    population_name=vasculature_population_name,
    realism_level=vasculature_realism_level, sql_filter=filter,
    representation=be.vascular_representation.OPTIMIZED_SEGMENT
)

### Astrocytes

In [ ]:
astrocytes_assembly_name = 'Astrocytes'
be.remove_assembly(astrocytes_assembly_name)
vasculature_assembly = be.add_assembly(astrocytes_assembly_name)
vasculature_model = be.add_astrocytes(
    assembly_name=astrocytes_assembly_name,
    population_name='astrocytes',
    vasculature_population_name=vasculature_population_name, # This creates end-feet to the vasculature
    population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    realism_level=morphology_realism_level, sql_filter=filter + ' AND guid%10=0'
)

### Rendering parameters

In [ ]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 500.0
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.epsilon_multiplier = 10.0
params.max_ray_depth = 1
params.show_background = False
params.main_exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
def set_materials(model_id, material_ids, palette, cast_user_data=False, reflection_index=0.0):
    user_params=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    glossinesses=list()
    emissions=list()
    cast_user_datas = list()
    
    for i in range(len(palette)):
        user_params.append(0.05)
        opacities.append(1.0)
        refraction_indices.append(1.0)
        reflection_indices.append(reflection_index)
        specular_exponents.append(5.0)
        glossinesses.append(0.1)
        c=palette[i]
        diffuse_colors.append(c)
        specular_colors.append(c                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        )
        shading_modes.append(be.SHADING_MODE_GOODSELL)
        emissions.append(0)
        cast_user_datas.append(cast_user_data)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        user_parameters=user_params,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices,
        cast_user_datas=cast_user_datas)

In [ ]:
def set_scene_materials():
    vasculature_info = be.get_vasculature_info(assembly_name=vasculature_assembly_name)
    vasculature_model_id = vasculature_info['modelId']    
    for model_id in be.get_model_ids()['ids']:
        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        if model_id == vasculature_model_id:
            palette = list()
            for _ in range(nb_materials):
                palette.append((0.8, 0.2, 0.1))
            set_materials(model_id=model_id, material_ids=material_ids, palette=palette, cast_user_data=False)
        else:
            palette = sns.color_palette('Blues', 10)
            p = list()
            for i in range(nb_materials):
                p.append(palette[5])
            set_materials(model_id=model_id, material_ids=material_ids, palette=p, cast_user_data=False)
    status = core.set_renderer()

In [ ]:
set_scene_materials()

In [ ]:
dx = 1000.0
dy = 10.0
be.add_box(
    name='Box',
    bottom_left_corner=Vector3(o.x - dx, o.y - dy, o.z - dx), 
    top_right_corner=Vector3(o.x + dx, o.y + dy, o.z + dx),
    color=Vector3(1,1,1))

## Snapshots

In [ ]:
status = core.set_camera(current='bio_explorer_perspective')

### Default overview

In [ ]:
status = core.set_camera(
    orientation = [-0.188, 0.28, 0.036, 0.940],
    position = [1577, 2183, 2511],
    target = [333, 1279, 646])

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)
mm.create_snapshot(
    renderer='advanced',
    path='/tmp', base_name='ngv_cylinder_4k_v1',
    size=[2160, 3840], samples_per_pixel=64)